# Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
Сложить собранные новости в БД

In [56]:
from pprint import pprint
from lxml import html
import requests
from pymongo import MongoClient

In [57]:

class News_mail:
    host = 'https://news.mail.ru/'
    headers = {

        'User-Agent':
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,  Gecko) Chrome/98.0.4758.82 Safari/537.36',
        
        'Accept-Charset': 'utf-8',
        'Accept-Language': 'ru'
    }

    def mail(self) -> list:
        news_list = []
        response = requests.get(self.host, headers=self.headers)
        dom = html.fromstring(response.text)
        news_elements = dom.xpath("//ul[@data-module='TrackBlocks']/li[@class='list__item']")
        for element in news_elements:
            news = {
                'title': str(element.xpath(".//text()")[0]),
                'url': str(element.xpath("./a/@href")[0])
            }
            news = self._extend(news)
            news_list.append(news)
        return news_list

    def _extend(self, news: dict) -> dict:
        response = requests.get(news['url'], headers=self.headers)
        dom = html.fromstring(response.text)
        news['source'] = str(dom.xpath("//a[contains(@class,'breadcrumbs__link')]//text()")[0])
        news['publication_date'] = str(dom.xpath("//span[@datetime]/@datetime")[0])
        return news
if __name__ == '__main__':
    news = News_mail()
    news_result = news.mail()
    pprint(news_result)    
client = MongoClient('127.0.0.1', 27017)
db = client['news_mail']
news_result += News_mail().mail()

db.news.insert_many(news_result)

pprint(news_result)     

[{'publication_date': '2022-02-28T11:40:13+03:00',
  'source': 'Известия',
  'title': 'Что\xa0изменится в\xa0жизни россиян с\xa01\xa0марта',
  'url': 'https://news.mail.ru/society/50235667/'},
 {'publication_date': '2022-02-28T15:39:48+03:00',
  'source': '© РИА Новости',
  'title': '«Ядерная триада» России начала дежурство усиленным составом',
  'url': 'https://news.mail.ru/politics/50245092/'},
 {'publication_date': '2022-02-28T15:53:45+03:00',
  'source': '© РИА Новости',
  'title': 'Путин назвал западное сообщество «империей лжи»',
  'url': 'https://news.mail.ru/politics/50245820/'},
 {'publication_date': '2022-02-28T17:35:10+03:00',
  'source': 'N + 1',
  'title': 'Физики создали рекордно красный материал',
  'url': 'https://news.mail.ru/society/50247840/'},
 {'publication_date': '2022-02-28T17:05:42+03:00',
  'source': 'РБК',
  'title': 'Что\xa0происходило с\xa0рублем и\xa0акциями после санкций '
           'против\xa0ЦБ',
  'url': 'https://news.mail.ru/economics/50247329/'},
 {'